In [1]:
!pip install segment-geospatial

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 13.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux201

In [2]:
import time
from typing import Tuple
import torch
from samgeo import SamGeo
from tqdm import tqdm
import google.colab.drive as drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [5]:
def samgeo_unit(
        sam: SamGeo,
        input_img_path: str,
        output_folder: str,
        batch: bool=True,
        foreground: bool=True,
        erosion_kernel: Tuple[int, int] = (3, 3),
        mask_multiplier: int =255
    ) -> str:

    output_path = output_folder + '/' + input_img_path.split('/')[-1].split('.')[0] + '_delineation.tif'
    sam.generate(
        input_img_path, output_path, batch=batch, foreground=foreground, erosion_kernel=erosion_kernel, mask_multiplier=mask_multiplier
    )

    output_vector_path = output_path[:-4] + '.gpkg'
    output_shp_path = output_path[:-4] + '.shp'

    sam.tiff_to_gpkg(output_path, output_vector_path, simplify_tolerance=None)
    sam.tiff_to_vector(output_path, output_shp_path)

    return output_vector_path

def segment_tiles(input_img_tiles: str, output_folder: str) -> None:
    img_list = input_img_tiles

    start = time.time()

    for idx, img in enumerate(tqdm(img_list)):
        print(f'Working on tile {idx}')

        output = samgeo_unit(img, output_folder)
        if torch.cuda.is_available:
            torch.cuda.empty_cache()

    end = time.time()
    print('Segmentations finished')
    print (f'Total time taken {(end-start)/60} mins')

In [6]:
sam_kwargs = {
    'points_per_side': 128,
    'pred_iou_thresh': 0.80,
    'stability_score_thresh': 0.95,
    'crop_n_layers': 1,
    'crop_n_points_downscale_factor': 2,
    'min_mask_region_area': 100,
}

sam = SamGeo(
    model_type='vit_h',
    checkpoint='sam_vit_h_4b8939.pth',
    sam_kwargs=sam_kwargs,
    device=device
)

Model checkpoint for vit_h not found.


Downloading...
From: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
To: /root/.cache/torch/hub/checkpoints/sam_vit_h_4b8939.pth
100%|██████████| 2.56G/2.56G [00:16<00:00, 152MB/s]
/usr/local/lib/python3.11/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any

In [7]:
input_img = '/content/drive/MyDrive/(4) Execute Stage /Rafid/samgeo_experiments/ground_truths/patch_1.tif'
export_folder = '/content/drive/MyDrive/(4) Execute Stage /Rafid/samgeo_experiments/validation'

In [8]:
_ = samgeo_unit(sam, input_img, export_folder)

100%|██████████| 25/25 [1:05:50<00:00, 158.04s/it]
